# Imports

In [1]:
pip install requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import defaultdict

# Années précédentes

In [24]:
years=[i for i in range(2010,2025)]
years

[2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024]

In [25]:
url_list=[f'https://fr.wikipedia.org/wiki/Miss_France_{year}' for year in years]
url_list

['https://fr.wikipedia.org/wiki/Miss_France_2010',
 'https://fr.wikipedia.org/wiki/Miss_France_2011',
 'https://fr.wikipedia.org/wiki/Miss_France_2012',
 'https://fr.wikipedia.org/wiki/Miss_France_2013',
 'https://fr.wikipedia.org/wiki/Miss_France_2014',
 'https://fr.wikipedia.org/wiki/Miss_France_2015',
 'https://fr.wikipedia.org/wiki/Miss_France_2016',
 'https://fr.wikipedia.org/wiki/Miss_France_2017',
 'https://fr.wikipedia.org/wiki/Miss_France_2018',
 'https://fr.wikipedia.org/wiki/Miss_France_2019',
 'https://fr.wikipedia.org/wiki/Miss_France_2020',
 'https://fr.wikipedia.org/wiki/Miss_France_2021',
 'https://fr.wikipedia.org/wiki/Miss_France_2022',
 'https://fr.wikipedia.org/wiki/Miss_France_2023',
 'https://fr.wikipedia.org/wiki/Miss_France_2024']

In [26]:
# Dictionnaire pour stocker les DataFrames
dataframes = {}

In [30]:
def extract_candidates_table(url, year):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Localiser la section "Candidates"
    section_candidates = None
    for header in soup.find_all('h2'):
        if "Candidates" in header.text:
            section_candidates = header
            break
    
    if section_candidates:
        # Trouver la table qui suit immédiatement cette section
        candidates_table = section_candidates.find_next('table', {'class': 'wikitable'})
        if candidates_table:
            # Extraire les en-têtes de la table
            headers = [header.text.strip() for header in candidates_table.find_all('th')]
            # Extraire les lignes de la table
            rows = []
            for row in candidates_table.find_all('tr')[1:]:  # Ignorer les en-têtes
                cols = [col.text.strip() for col in row.find_all('td')]
                rows.append(cols)
            # Créer un DataFrame pandas
            df = pd.DataFrame(rows, columns=headers)
            df['année'] = year
            return df
    return None

In [31]:
for url in url_list:
    year = url.split('_')[-1]  # Extraire l'année de l'URL
    df = extract_candidates_table(url, year)
    if df is not None:
        dataframes[f"année {year}"] = df
        print(f"Données extraites pour l'année {year}.")
    else:
        print(f"Pas de table 'Candidates' trouvée pour l'année {year}.")

Données extraites pour l'année 2010.
Données extraites pour l'année 2011.
Données extraites pour l'année 2012.
Données extraites pour l'année 2013.
Données extraites pour l'année 2014.
Données extraites pour l'année 2015.
Données extraites pour l'année 2016.
Données extraites pour l'année 2017.
Données extraites pour l'année 2018.
Données extraites pour l'année 2019.
Données extraites pour l'année 2020.
Données extraites pour l'année 2021.
Données extraites pour l'année 2022.
Données extraites pour l'année 2023.
Données extraites pour l'année 2024.


In [29]:
print(dataframes.keys())

dict_keys(['année 2010', 'année 2011', 'année 2012', 'année 2013', 'année 2014', 'année 2015', 'année 2016', 'année 2017', 'année 2018', 'année 2019', 'année 2020', 'année 2021', 'année 2022', 'année 2023', 'année 2024'])


In [35]:
categories = defaultdict(list)

# Identifier les headers de chaque DataFrame et les regrouper
for year, df in dataframes.items():
    headers = tuple(df.columns)  # Les colonnes de chaque DataFrame sous forme de tuple
    categories[headers].append(year)

# Afficher les catégories et les années associées
for idx, (header_set, years) in enumerate(categories.items(), start=1):
    print(f"Catégorie {idx}:")
    print("Headers :", header_set)
    print("Années :", years)
    print("-" * 50)


Catégorie 1:
Headers : ('Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue à...', 'année')
Années : ['année 2010']
--------------------------------------------------
Catégorie 2:
Headers : ('Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Classement final', 'année')
Années : ['année 2011', 'année 2014', 'année 2015', 'année 2016', 'année 2021']
--------------------------------------------------
Catégorie 3:
Headers : ('Région', 'Nom[2]', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Classement final', 'année')
Années : ['année 2012', 'année 2013']
--------------------------------------------------
Catégorie 4:
Headers : ('Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Résultat', 'année')
Années : ['année 2017']
--------------------------------------------------
Catégorie 5:
Headers : ('région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le', 'Groupe', 'Classement final', 'année')
Années : ['année 2018']
--------------------------------------------------
Catégorie 6:


# Nettoyage des colonnes

In [ ]:
def clean_columns(df):
    # Nettoyer les noms des colonnes en supprimant les crochets et leur contenu
    df.columns = df.columns.str.replace(r'\[.*?\]', '', regex=True).str.strip()
    # Supprimer la colonne 'Réf.' si elle existe
    if 'Réf.' in df.columns:
        df = df.drop(columns=['Réf.'])
    return df

for year, df in dataframes.items():
    dataframes[year] = clean_columns(df)

for year, df in dataframes.items():
    print(f"Année : {year}")
    print(df.columns)
    print("-" * 50)

Année : année 2010
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue à...', 'année'], dtype='object')
--------------------------------------------------
Année : année 2011
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le',
       'Classement final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2012
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le',
       'Classement final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2013
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le',
       'Classement final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2014
Index(['Région', 'Nom', 'Âge', 'Taille', 'Résidence', 'Élue le',
       'Classement final', 'année'],
      dtype='object')
--------------------------------------------------
Année : année 2015
Index(['Région', 'Nom', 'Âge', 'Taille', '

# URL de la page Wikipédia de Miss France 2024

In [ ]:
url = "https://fr.wikipedia.org/wiki/Miss_France_2024"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

sections = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

for section in sections:
    level = section.name  # Niveau de la section (h1, h2, etc.)
    title = section.get_text(strip=True)  # Texte de la section
    print(f"{level}: {title}")


h2: Sommaire
h1: Miss France 2024
h2: Classement final
h2: Ordre d'annonce des finalistes
h3: Top 15
h3: Top 5
h2: Déroulement
h3: Hommage à Geneviève de Fontenay
h2: Préparation
h3: Voyage de préparation
h2: Jury
h2: Candidates
h2: Classement
h3: Résultats du Top 15
h4: Résultats du Top 5
h3: Prix attribués
h2: Observations
h3: Notes sur les candidates
h3: Notes sur le classement des régions
h2: Audiences
h2: Notes et références
h3: Notes
h3: Références


In [14]:
section_candidates = None
for header in soup.find_all('h2'):
    if "Candidates" in header.text:
        section_candidates = header
        print('Section Trouvée')
        break

Section Trouvée


In [20]:
if section_candidates:
    candidates_table = section_candidates.find_next('table', {'class': 'wikitable'})
else :
    print('Pas de table trouvée')

In [18]:
candidates_table

<table class="wikitable sortable">
<caption>
</caption>
<tbody><tr>
<th scope="col">Région
</th>
<th scope="col">Nom
</th>
<th scope="col">Âge<sup class="reference" id="cite_ref-21"><a href="#cite_note-21"><span class="cite_crochet">[</span>N 1<span class="cite_crochet">]</span></a></sup>
</th>
<th scope="col">Taille
</th>
<th scope="col">Résidence
</th>
<th scope="col">Élue le
</th>
<th scope="col">Classement final
</th>
<th><abbr class="abbr" title="Références">Réf.</abbr>
</th></tr>
<tr>
<td><span class="flagicon" data-sort-value=""><span class="mw-image-border noviewer" typeof="mw:File"><a class="mw-file-description" href="/wiki/Fichier:Flag_of_Alsace_(old).svg" title="Drapeau de l'Alsace"><img alt="Drapeau de l'Alsace" class="mw-file-element" data-file-height="261" data-file-width="390" decoding="async" height="13" src="//upload.wikimedia.org/wikipedia/commons/thumb/f/f1/Flag_of_Alsace_%28old%29.svg/20px-Flag_of_Alsace_%28old%29.svg.png" srcset="//upload.wikimedia.org/wikipedia/co

In [21]:
if candidates_table:
    headers = [header.text.strip() for header in candidates_table.find_all('th')]

    rows = []
    for row in candidates_table.find_all('tr')[1:]:  # Ignorer les en-têtes
        cols = [col.text.strip() for col in row.find_all('td')]
        rows.append(cols)

    # Créer un DataFrame pandas
    df_candidates = pd.DataFrame(rows, columns=headers)

    # Sauvegarder dans un fichier CSV et afficher un aperçu
    df_candidates.to_csv('candidates_miss_france_2024.csv', index=False)
    print("Tableau des candidates extrait avec succès !")
    print(df_candidates.head())
else:
    print("Aucune table trouvée dans la section 'Candidates'.")

Tableau des candidates extrait avec succès !
           Région             Nom Âge[N 1]  Taille                Résidence  \
0     Miss Alsace  Adeline Vetter   27 ans  1,71 m                 Rossfeld   
1  Miss Aquitaine     Lola Turpin   19 ans  1,73 m                Trélissac   
2   Miss Auvergne   Oriane Mallet   22 ans  1,83 m                    Vichy   
3  Miss Bourgogne   Luna Lacharme   18 ans  1,75 m  La Chapelle-de-Guinchay   
4   Miss Bretagne  Noémie Le Bras   21 ans  1,75 m       Le Cloître-Pleyben   

                                Élue le  \
0        10 septembre 2023 à Kirrwiller   
1           3 septembre 2023 à Bordeaux   
2  24 septembre 2023 à Vals-près-le-Puy   
3     8 octobre 2023 à Chalon-sur-Saône   
4          29 septembre 2023 à Ploemeur   

                                    Classement final       Réf.  
0                    Prix de la culture générale[21]  [22],[23]  
1  Prix de la camaraderie[24] et  prix de l’éloqu...  [26],[27]  
2                     P

In [22]:
df_candidates

,Région,Nom,Âge[N 1],Taille,Résidence,Élue le,Classement final,Réf.
0,Miss Alsace,Adeline Vetter,27 ans,"1,71 m",Rossfeld,10 septembre 2023 à Kirrwiller,Prix de la culture générale[21],"[22],[23]"
1,Miss Aquitaine,Lola Turpin,19 ans,"1,73 m",Trélissac,3 septembre 2023 à Bordeaux,Prix de la camaraderie[24] et prix de l’éloqu...,"[26],[27]"
2,Miss Auvergne,Oriane Mallet,22 ans,"1,83 m",Vichy,24 septembre 2023 à Vals-près-le-Puy,"Prix de l’aventurière[25],[28]","[29],[30]"
3,Miss Bourgogne,Luna Lacharme,18 ans,"1,75 m",La Chapelle-de-Guinchay,8 octobre 2023 à Chalon-sur-Saône,Top 15 (10e),"[31],[32]"
4,Miss Bretagne,Noémie Le Bras,21 ans,"1,75 m",Le Cloître-Pleyben,29 septembre 2023 à Ploemeur,,"[33],[34]"
5,Miss Centre-Val de Loire,Emmy Gisclon,22 ans,"1,76 m",Chambray-les-Tours,17 septembre 2023 à Dreux,Top 15 (13e),"[35],[36]"
6,Miss Champagne-Ardenne,Noa Dutitre,22 ans,"1,70 m",Reims,6 octobre 2023 à Tinqueux,,"[37],[38]"
7,Miss Corse,Sandra Bak,23 ans,"1,71 m",Ajaccio,28 juillet 2023 à Porticcio,Top 15 (14e),"[39],[40]"
8,Miss Côte d'Azur,Karla Bchir,19 ans,"1,75 m",Cannes,30 juillet 2023 à Fréjus,Top 15 (12e),"[41],[42]"
9,Miss Franche-Comté,Sonia Coutant,24 ans,"1,72 m",Champagnole,20 octobre 2023 à Dole,Prix du costume régional[43],"[44],[45]"
